In [ ]:
import pandas as pd

# Cargar archivo
df = pd.read_csv(r"C:\Users\nono_\AppRegresion\data\autos_limpios.csv")


# Ver columnas únicas si querés explorar
print(df.columns)

import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignorar solo las advertencias de convergencia
warnings.filterwarnings("ignore", category=ConvergenceWarning)

Index(['marca', 'modelo', 'ano', 'color', 'tipo de combustible', 'puertas',
       'transmision', 'motor', 'tipo de carroceria', 'fecha publicacion',
       'kilometros', 'precio', 'traccion'],
      dtype='object')


In [15]:
'''import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suprimir los warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
multiplicador = 5
# Función para detectar outliers por modelo Y año usando el método IQR
def detectar_outliers_por_modelo_y_ano(df):
    # Crear una copia para evitar advertencias
    df_resultado = df.copy()
    df_resultado['es_outlier'] = False
    
    # Obtener modelos únicos
    modelos = df['modelo'].unique()
    
    for modelo in modelos:
        # Filtrar por modelo
        df_modelo = df[df['modelo'] == modelo]
        
        # Obtener años para este modelo
        anos = df_modelo['ano'].unique()
        
        for ano in anos:
            # Filtrar por modelo y año
            grupo = df_modelo[df_modelo['ano'] == ano]
            
            # Solo aplicar si hay suficientes datos para este modelo y año
            if len(grupo) >= 4:  # Necesitamos suficientes puntos para calcular outliers
                q1 = grupo['precio'].quantile(0.25)
                q3 = grupo['precio'].quantile(0.75)
                iqr = q3 - q1
                limite_inferior = q1 - multiplicador * iqr
                limite_superior = q3 + multiplicador * iqr
                
                # Identificar outliers para este modelo y año
                indices_outliers = grupo[(grupo['precio'] < limite_inferior) | 
                                         (grupo['precio'] > limite_superior)].index
                
                # Marcar en el dataframe resultado
                df_resultado.loc[indices_outliers, 'es_outlier'] = True
    
    return df_resultado

# Configurar el tamaño de la figura según la cantidad de modelos
modelos = df['modelo'].unique()
n_modelos = len(modelos)
fig, axes = plt.subplots(nrows=n_modelos, figsize=(14, 6*n_modelos))

# Si solo hay un modelo, convertir axes en una lista
if n_modelos == 1:
    axes = [axes]

# Aplicar detección de outliers a todo el dataframe
df_con_outliers = detectar_outliers_por_modelo_y_ano(df)

# Crear un gráfico para cada modelo
for i, modelo in enumerate(modelos):
    # Filtrar datos para el modelo actual
    datos_modelo = df_con_outliers[df_con_outliers['modelo'] == modelo].copy()
    
    # Obtener los años específicos para este modelo
    anos_modelo = sorted(datos_modelo['ano'].unique())
    
    # Separar outliers y datos normales
    outliers = datos_modelo[datos_modelo['es_outlier']].copy()
    datos_normales = datos_modelo[~datos_modelo['es_outlier']].copy()
    
    # Convertir precios a millones
    datos_normales.loc[:, 'precio_millones'] = datos_normales['precio'] / 1000000
    
    # Graficar los datos normales en azul
    axes[i].scatter(datos_normales['ano'], datos_normales['precio_millones'], 
                   color='blue', alpha=0.7, s=70, label='Datos normales')
    
    # Graficar los outliers en rojo (solo si hay outliers)
    if not outliers.empty:
        outliers.loc[:, 'precio_millones'] = outliers['precio'] / 1000000
        axes[i].scatter(outliers['ano'], outliers['precio_millones'], 
                       color='red', alpha=0.7, s=70, label='Outliers')
        
        # Añadir anotaciones para los outliers
        for idx, row in outliers.iterrows():
            axes[i].annotate(f'{row["ano"]}: {row["precio_millones"]:.2f}M', 
                            (row['ano'], row['precio_millones']),
                            xytext=(5, 5), textcoords='offset points',
                            fontsize=9, color='darkred')
    
    # Configurar el eje X para mostrar los años de este modelo
    axes[i].set_xticks(anos_modelo)
    axes[i].tick_params(axis='x', rotation=45)
    
    # Añadir títulos y etiquetas
    axes[i].set_title(f'Modelo: {modelo}', fontsize=18, fontweight='bold')
    axes[i].set_xlabel('Año', fontsize=14)
    axes[i].set_ylabel('Precio (millones)', fontsize=14)
    
    # Formatear el eje Y para mostrar los valores con 2 decimales
    axes[i].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))
    
    # Añadir leyenda y mejorar visualización
    axes[i].legend(fontsize=12)
    axes[i].grid(True, linestyle='--', alpha=0.7)
    sns.despine(ax=axes[i])

# Ajustar espaciado entre subplots
plt.tight_layout(pad=4.0)
plt.show()'''

'import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport warnings\n\n# Suprimir los warnings\nwarnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)\nmultiplicador = 5\n# Función para detectar outliers por modelo Y año usando el método IQR\ndef detectar_outliers_por_modelo_y_ano(df):\n    # Crear una copia para evitar advertencias\n    df_resultado = df.copy()\n    df_resultado[\'es_outlier\'] = False\n    \n    # Obtener modelos únicos\n    modelos = df[\'modelo\'].unique()\n    \n    for modelo in modelos:\n        # Filtrar por modelo\n        df_modelo = df[df[\'modelo\'] == modelo]\n        \n        # Obtener años para este modelo\n        anos = df_modelo[\'ano\'].unique()\n        \n        for ano in anos:\n            # Filtrar por modelo y año\n            grupo = df_modelo[df_modelo[\'ano\'] == ano]\n            \n            # Solo aplicar si hay suficientes datos para este modelo y año\n            if l

In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def data_point_remover(df, x_col='ano', y_col='kilometros', model_col='modelo'):
    """
    Dashboard interactivo para visualizar y eliminar outliers en la columna 'kilometros',
    agrupando por modelo y año.
    """
    # Hacer copias del DataFrame
    df_original = df.copy()
    df_working = df.copy()
    removed_points = []

    # Obtener lista de modelos únicos
    models = sorted(df[model_col].unique())

    # Widgets informativos
    info = widgets.HTML("<h3>Eliminador de Outliers en Kilómetros</h3><p>Haz clic en cualquier punto para eliminarlo.</p>")
    status = widgets.HTML(f"<b>Estado:</b> {len(df_working)} registros, 0 eliminados")

    # Dropdown para seleccionar modelo
    model_select = widgets.Dropdown(
        options=models,
        value=models[0] if models else None,
        description='Modelo:',
        style={'description_width': 'initial'}
    )

    # Área de salida para el gráfico
    plot_output = widgets.Output()
    message = widgets.Output()

    def detect_outliers_and_thresholds(model_df):
        df_res = model_df.copy()
        df_res['es_outlier'] = False
        thresholds = []

        for year in sorted(df_res[x_col].unique()):
            year_data = df_res[df_res[x_col] == year]
            lower, upper = np.nan, np.nan
            if len(year_data) >= 4:
                q1 = year_data[y_col].quantile(0.25)
                q3 = year_data[y_col].quantile(0.75)
                iqr = q3 - q1
                lower = q1 - 1.5 * iqr
                upper = q3 + 1.5 * iqr
                df_res.loc[year_data.index, 'es_outlier'] = (
                    (year_data[y_col] < lower) | (year_data[y_col] > upper)
                )
            thresholds.append({'year': year, 'lower_bound': lower, 'upper_bound': upper})

        return df_res, pd.DataFrame(thresholds)

    def update_plot():
        plot_output.clear_output(wait=True)
        selected = model_select.value
        data = df_working[df_working[model_col] == selected].copy()

        if data.empty:
            with plot_output:
                print(f"No hay datos para el modelo {selected}")
            return

        data, thr = detect_outliers_and_thresholds(data)
        fig = go.Figure()

        # Datos normales
        normal = data[~data['es_outlier']]
        fig.add_trace(go.Scatter(
            x=normal[x_col], y=normal[y_col], mode='markers',
            marker=dict(color='blue', size=8), name='Normal',
            customdata=normal.index,
            hovertemplate='Año: %{x}<br>Kilómetros: %{y}<br>ID: %{customdata}<extra></extra>'
        ))

        # Outliers
        out = data[data['es_outlier']]
        if not out.empty:
            fig.add_trace(go.Scatter(
                x=out[x_col], y=out[y_col], mode='markers',
                marker=dict(color='red', size=10, symbol='circle-open'), name='Outlier',
                customdata=out.index,
                hovertemplate='Año: %{x}<br>Kilómetros: %{y}<br>ID: %{customdata}<extra></extra>'
            ))

        # Umbrales
        fig.add_trace(go.Scatter(
            x=thr['year'], y=thr['upper_bound'], mode='lines',
            line=dict(color='red', width=2, dash='dash'), name='Umbral Superior',
            hovertemplate='Año: %{x}<br>Umbral: %{y}<extra></extra>'
        ))
        fig.add_trace(go.Scatter(
            x=thr['year'], y=thr['lower_bound'], mode='lines',
            line=dict(color='red', width=2, dash='dash'), name='Umbral Inferior',
            hovertemplate='Año: %{x}<br>Umbral: %{y}<extra></extra>'
        ))

        fig.update_layout(
            title=f"Modelo: {selected}", xaxis_title='Año', yaxis_title='Kilómetros',
            height=500, width=800, clickmode='event+select'
        )
        fig.update_xaxes(tickmode='array', tickvals=thr['year'], ticktext=thr['year'].astype(str))

        with plot_output:
            figw = go.FigureWidget(fig)

            def on_click(trace, points, selector):
                nonlocal df_working, removed_points
                if points.point_inds:
                    idx = trace.customdata[points.point_inds[0]]
                    removed_points.append(idx)
                    df_working = df_working.drop(idx)
                    status.value = f"<b>Estado:</b> {len(df_working)} registros, {len(removed_points)} eliminados"
                    update_plot()

            for tr in figw.data[:2]:
                tr.on_click(on_click)
            display(figw)

    # Callbacks y botones
    model_select.observe(lambda change: update_plot() if change['name']=='value' else None, names='value')

    def remove_all_outliers(b):
        nonlocal df_working, removed_points
        selected = model_select.value
        subset = df_working[df_working[model_col] == selected]
        det, _ = detect_outliers_and_thresholds(subset)
        idxs = det[det['es_outlier']].index.tolist()
        if idxs:
            removed_points.extend(idxs)
            df_working = df_working.drop(idxs)
            status.value = f"<b>Estado:</b> {len(df_working)} registros, {len(removed_points)} eliminados"
            with message:
                clear_output()
                print(f"Eliminados {len(idxs)} outliers de {selected}")
            update_plot()
        else:
            with message:
                clear_output()
                print(f"No hay outliers en {selected}")

    def undo_last(b):
        nonlocal df_working, removed_points
        if removed_points:
            last = removed_points.pop()
            df_working = pd.concat([df_working, df_original.loc[[last]]])
            status.value = f"<b>Estado:</b> {len(df_working)} registros, {len(removed_points)} eliminados"
            update_plot()

    def reset_all(b):
        nonlocal df_working, removed_points
        df_working = df_original.copy()
        removed_points = []
        status.value = f"<b>Estado:</b> {len(df_working)} registros, 0 eliminados"
        update_plot()
        with message:
            clear_output()

    def save_data(b):
        nonlocal df_working, df_original
        clear_output()
        print(f"DataFrame guardado con {len(df_working)} filas ({len(df_original)-len(df_working)} eliminados)")
        global df
        df = df_working.copy()
        global clean_df
        clean_df = df_working.copy()

    # Botones
    btns = widgets.HBox([
        widgets.Button(description='Deshacer', icon='undo', on_click=undo_last),
        widgets.Button(description='Reiniciar', icon='refresh', on_click=reset_all),
        widgets.Button(description='Eliminar Outliers', icon='trash', button_style='danger', on_click=remove_all_outliers),
        widgets.Button(description='Guardar', icon='save', on_click=save_data)
    ])

    # Navegación entre modelos
    def prev_model(b):
        idx = models.index(model_select.value)
        if idx>0: model_select.value = models[idx-1]
    def next_model(b):
        idx = models.index(model_select.value)
        if idx<len(models)-1: model_select.value = models[idx+1]
    nav = widgets.HBox([widgets.Button(description='←', on_click=prev_model), model_select, widgets.Button(description='→', on_click=next_model)])

    dashboard = widgets.VBox([info, status, nav, btns, plot_output, message])

    update_plot()
    dashboard.get_clean_df = lambda: clean_df if 'clean_df' in globals() else df_working.copy()
    return dashboard


In [ ]:
# Crear el dashboard (con el nombre correcto de la función)
dashboard = data_point_remover(df)

# Mostrar el dashboard
display(dashboard)

# Después de eliminar outliers y hacer clic en "Guardar":
clean_df = dashboard.get_clean_df()
df=clean_df

In [18]:
df.describe()

,ano,puertas,motor,fecha publicacion,kilometros,precio
count,50272.000000,50272.000000,50272.000000,50272.000000,5.027200e+04,5.027200e+04
mean,2016.398433,3.922939,1.763851,52.095540,1.062795e+05,2.018564e+07
std,4.812234,0.558087,0.510098,66.650631,5.991688e+04,1.085966e+07
min,1970.000000,2.000000,1.000000,1.000000,3.000000e+01,4.000000e+06
25%,2014.000000,4.000000,1.500000,12.000000,6.600000e+04,1.350000e+07
50%,2017.000000,4.000000,1.600000,28.000000,1.000000e+05,1.750000e+07
75%,2020.000000,4.000000,1.800000,60.000000,1.390000e+05,2.366100e+07
max,2024.000000,5.000000,6.400000,365.000000,1.111111e+06,1.493750e+08
